# Dose Metrics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AustralianCancerDataNetwork/pydicer/blob/main/examples/DoseMetrics.ipynb)

In this example notebook we will compute [Dose Volume Histograms (DVH)](https://pyplati.github.io/platipy/dose.html#module-platipy.imaging.dose.dvh) for our `RTDOSE` objects
across structures found in `RTSTRUCT` objects in our dataset. We use
[HNSCC](https://wiki.cancerimagingarchive.net/display/Public/HNSCC) data from the Cancer Imaging
Archive which has already been converted using PyDicer for demonstration purposes.

In [1]:
try:
    from pydicer import PyDicer
except ImportError:
    !pip install pydicer
    from pydicer import PyDicer

from pathlib import Path

from pydicer.utils import fetch_converted_test_data

## Fetch data

HNSCC data prepared for this example are downloaded and stored into a `testdata_hnscc` directory.
We will use this for our PyDicer working directory.

In [2]:
working_directory = fetch_converted_test_data("./testdata_hnscc", dataset="HNSCC")

Working directory %s aready exists, won't download test data.


## Initialise PyDicer object

Using the working directory containing the test data.

In [3]:
pydicer = PyDicer(working_directory)

## Compute DVH

Before we can extract dose metrics, we must compute Dose Volume Histograms for all dose objects and
structure sets. This is done using the [compute_dvh](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.compute_dvh) function.

In [4]:
pydicer.analyse.compute_dvh()

  0%|          | 0/4 [00:00<?, ?objects/s, Compute DVH]

 25%|██▌       | 1/4 [00:12<00:36, 12.19s/objects, Compute DVH]

 50%|█████     | 2/4 [00:22<00:22, 11.35s/objects, Compute DVH]

 75%|███████▌  | 3/4 [00:36<00:12, 12.22s/objects, Compute DVH]

100%|██████████| 4/4 [00:45<00:00, 11.23s/objects, Compute DVH]

100%|██████████| 4/4 [00:45<00:00, 11.48s/objects, Compute DVH]

## Inspect DVH

DVHs computed are stored in the respective dose object directories on the file system. Inspect a
dose object directory (e.g. `testdata_hnscc/data/HNSCC-01-0019/doses/309e1a`). Here you will find
a `.png` file which plots the DVH for each of the linked structures. In addition a `.csv` file 
stores the raw DVH values.

The DVHs can for this dataset can be loaded into a pandas DataFrame with the
[get_all_dvhs_for_dataset](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.get_all_dvhs_for_dataset) function.

In [5]:
df_dvh = pydicer.analyse.get_all_dvhs_for_dataset()
df_dvh.head()

,patient,struct_hash,dose_hash,label,cc,mean,0.0,0.1,0.2,0.3,...,77.2,77.3,77.4,77.5,77.6,77.7,77.8,77.9,78.0,78.1
0,HNSCC-01-0019,7cdcd9,309e1a,+1,4640.553474,29.679710,1.0,0.837845,0.835239,0.83267,...,0.000028,0.000020,0.000009,0.000002,6.165262e-07,0.0,0.0,0.0,0.0,0.0
1,HNSCC-01-0019,7cdcd9,309e1a,-.3,2689.948082,43.784290,1.0,1.000000,1.000000,1.00000,...,0.000048,0.000035,0.000016,0.000004,1.063598e-06,0.0,0.0,0.0,0.0,0.0
2,HNSCC-01-0019,7cdcd9,309e1a,Brain,549.576759,24.043829,1.0,1.000000,1.000000,1.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0
3,HNSCC-01-0019,7cdcd9,309e1a,Brainstem,47.636032,39.998913,1.0,1.000000,1.000000,1.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0
4,HNSCC-01-0019,7cdcd9,309e1a,CTV63,467.602730,71.274086,1.0,1.000000,1.000000,1.00000,...,0.000275,0.000202,0.000092,0.000024,6.118491e-06,0.0,0.0,0.0,0.0,0.0


## Compute Dose Metrics

The [compute_dose_metrics](https://australiancancerdatanetwork.github.io/pydicer/analyse.html#pydicer.analyse.data.AnalyseData.compute_dose_metrics) function in the `analyse` module can compute **D**, **V** and **Dcc**
metrics. Specify the points at which to compute those values. For example, the following cell
computes the **D95**, **D50**, **V5** and **Dcc10**.

In [6]:
df_dose_metrics = pydicer.analyse.compute_dose_metrics(
    d_point=[95, 50],
    v_point=[5],
    d_cc_point=[10]
)
df_dose_metrics.head()

,patient,struct_hash,dose_hash,label,cc,mean,D95,D50,V5,D10cc
0,HNSCC-01-0019,7cdcd9,309e1a,+1,4640.553474,29.679710,0.030835,25.358893,3558.228493,75.501190
1,HNSCC-01-0019,7cdcd9,309e1a,-.3,2689.948082,43.784290,11.384142,43.358867,2668.971062,75.501190
2,HNSCC-01-0019,7cdcd9,309e1a,Brain,549.576759,24.043829,7.657684,22.339093,544.220924,56.311918
3,HNSCC-01-0019,7cdcd9,309e1a,Brainstem,47.636032,39.998913,28.935185,39.202222,47.636032,46.232806
4,HNSCC-01-0019,7cdcd9,309e1a,CTV63,467.602730,71.274086,66.177108,71.589748,467.602730,75.494366
